In [2]:
import pandas as pd
import os

In [3]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# -------------------- DEFINE CUSTOM TEMPLATE -------------------- #
pio.templates['draft'] = go.layout.Template(layout=dict(
    margin=dict(l=50, r=50, b=50, t=50),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
)
))
pio.templates.default = "plotly+draft"

### Fragment Results

In [13]:
fragment_result = pd.read_csv('./Pos_training/fragment_results.csv')
fragment_result = pd.read_csv('./Pos_training/WESAD_no_ecg_fragment_both.csv')
fragment_result.head(5)

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,acc,acc_std,f1,f1_std,recall,recall_std,precision,precision_std
0,TSTCC,MLP,linearprobing,one_domain,Fragment,0.5509,0.5317,0.0063,0.4072,0.0076,0.5509,0.0080,1.1092,0.0158
1,TSTCC,MLP,linearprobing,four_domains,Fragment,0.5043,0.4837,0.0066,0.3457,0.0084,0.5043,0.0051,1.0121,0.0300
2,TSTCC,FCN,finetuning,four_domains,Fragment,0.2810,0.1680,0.1165,0.1032,0.0564,0.2810,0.1058,1.8616,0.1345
3,TSTCC,FCN,linearprobing,one_domain,Fragment,0.1951,0.1054,0.0496,0.0825,0.0337,0.1951,0.0570,1.6465,0.0328
4,TSTCC,FCN,linearprobing,four_domains,Fragment,0.1988,0.0939,0.0192,0.1003,0.0306,0.1988,0.0192,1.6779,0.0777


In [14]:
fig = px.bar(fragment_result, x='ssl_model', y='f1', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group',
             labels={'ssl_model': 'SSL Model', 'acc': 'Accuracy', 'scenario': 'Scenario'},)

fig.update_yaxes(range=[0, 1])
fig.update_layout(height=640, width=1000, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': ('Scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['TSTCC', 'TS2Vec', 'Series2Vec'], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': 'yJi7lpAP2j+PU3Qkl//gPwfOGVHaG9A/', 'dtype': 'f8'},
              'yaxis': 'y3'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('Scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['TSTCC', 'TS2Vec', 'Series2Vec'], dtype=object),
              'xaxis': 'x4',
              'y': {'bdata': 'H4XrUbgetT/FILByaJG9P4V80LNZ9bk/', 'dtype': 'f8'},
              'yaxis': 'y4'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('Scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['TSTCC', 'TS2Vec', 'Series2Vec'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'ldQJaCJsqD8IPZtVn6vjP5p3nKIjubw/', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('Scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['TSTCC', 'TS2Vec', 'Series2Vec'], dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': '1sVtNIC3sD90RpT2Bl+4P44G8BZIUKw/', 'dtype': 'f8'},
              'yaxis': 'y2'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('Scenario=four_domains<br>calib' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'four_domains',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'four_domains',
              'offsetgroup': 'four_domains',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['TSTCC', 'TS2Vec', 'Series2Vec'], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': 'coqO5PIf1j+At0CC4sfgP2ZmZmZmZto/', 'dtype': 'f8'},
              'yaxis': 'y3'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('Scenario=four_domains<br>calib' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'four_domains',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'four_domains',
              'offsetgroup': 'four_domains',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['TSTCC', 'TS2Vec', 'Series2Vec'], dtyp

### IEEEPPG Results

In [ ]:
ieeeppg_result = pd.read_csv('./Pos_training/ieeeppg_results.csv')
ieeeppg_result.head()

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,mse,mse_std,r2_score,r2_score_std,rmse,rmse_std,mae,mae_std
0,TSTCC,MLP,linearprobing,one_domain,IEEEPPG,517.3157,0.0101,0.0043,22.7445,0.0498,18.3980,0.1768,529.4080,1.4312
1,TSTCC,MLP,linearprobing,four_domains,IEEEPPG,518.3404,0.0082,0.0030,22.7671,0.0346,18.4338,0.1247,529.0043,1.1918
2,TSTCC,MLP,finetuning,one_domain,IEEEPPG,535.3663,-0.0244,0.0264,23.1364,0.2993,19.1758,0.5177,543.2826,11.4943
3,TSTCC,MLP,finetuning,four_domains,IEEEPPG,604.8880,-0.1574,0.0264,24.5932,0.2804,21.0037,0.3759,591.9205,15.9592
4,TSTCC,FCN,linearprobing,one_domain,IEEEPPG,1190.1912,-1.2774,0.2310,34.4638,1.7447,30.7696,1.6269,1177.3956,119.0651


In [ ]:
fig = px.bar(ieeeppg_result, x='ssl_model', y='rmse', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group')

fig.update_layout(height=640, width=1000, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

### Fragment

In [4]:
ieeeppg_result = pd.read_csv('./Pos_training/WESAD_no_ecg_fragment.csv')
ieeeppg_result.head(50)

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,acc,acc_std,f1,f1_std,recall,recall_std,precision,precision_std
0,S2V,MLP,linearprobing,one_domain,Fragment,0.3387,0.3397,0.0073,0.2499,0.0071,0.3387,0.0080,1.6643,0.0418
1,S2V,FCN,linearprobing,one_domain,Fragment,0.1939,0.1375,0.0285,0.0828,0.0177,0.1939,0.0422,1.6122,0.0187
2,S2V,MLP,finetuning,one_domain,Fragment,0.2172,0.1367,0.0837,0.1634,0.0397,0.2172,0.0703,8.3586,2.6819
3,S2V,FCN,finetuning,one_domain,Fragment,0.1215,0.0531,0.0174,0.0775,0.0222,0.1215,0.0146,1.8672,0.0215
4,TS2V,MLP,finetuning,one_domain,Fragment,0.6859,0.6872,0.0310,0.6305,0.0387,0.6859,0.0323,1.5748,0.1744
5,TS2V,MLP,linearprobing,one_domain,Fragment,0.6172,0.6213,0.0115,0.5345,0.0171,0.6172,0.0070,1.0728,0.0226
6,TS2V,FCN,linearprobing,one_domain,Fragment,0.2675,0.1854,0.0342,0.1169,0.0166,0.2675,0.0649,1.6208,0.0580
7,TS2V,FCN,finetuning,one_domain,Fragment,0.1656,0.0849,0.0622,0.0942,0.0603,0.1656,0.0869,2.3064,0.7688
8,TSTCC,MLP,linearprobing,one_domain,Fragment,0.5865,0.5451,0.0297,0.4243,0.0348,0.5865,0.0367,0.8862,0.0152
9,TSTCC,FCN,linearprobing,one_domain,Fragment,0.2405,0.1909,0.0180,0.1967,0.0291,0.2405,0.0265,1.6382,0.0360


In [5]:
ieeeppg_result.columns

Index(['ssl_model', 'head_model', 'calibration', 'scenario', 'dataset',
       'avg_loss', 'acc', 'acc_std', 'f1', 'f1_std', 'recall', 'recall_std',
       'precision', 'precision_std'],
      dtype='object')

In [8]:
fig = px.bar(ieeeppg_result, x='ssl_model', y='f1', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group')



fig.update_yaxes(range=[0, 1])
fig.update_layout(height=640, width=1000, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': ('scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['S2V', 'TS2V', 'TSTCC'], dtype=object),
              'xaxis': 'x3',
              'y': {'bdata': 'eJyiI7n8zz+0yHa+nxrhP9ZW7C+7J9s/', 'dtype': 'f8'},
              'yaxis': 'y3'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['S2V', 'TS2V', 'TSTCC'], dtype=object),
              'xaxis': 'x4',
              'y': {'bdata': 'UdobfGEytT+wA+eMKO29PwXFjzF3Lck/', 'dtype': 'f8'},
              'yaxis': 'y4'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['S2V', 'TS2V', 'TSTCC'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'mEwVjErqxD+TGARWDi3kP+F6FK5H4ao/', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('scenario=one_domain<br>calibra' ... '%{x}<br>f1=%{y}<extra></extra>'),
              'legendgroup': 'one_domain',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'one_domain',
              'offsetgroup': 'one_domain',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['S2V', 'TS2V', 'TSTCC'], dtype=object),
              'xaxis': 'x2',
              'y': {'bdata': '16NwPQrXsz/Mf0i/fR24P8WPMXctIa8/', 'dtype': 'f8'},
              'yaxis': 'y2'}],
    'layout': {'annotations': [{'font': {},
                                'showarrow': False,
                                'text': 'head_model=MLP',
                                'x': 0.24,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {},
                                'showarrow': False,
                                'text': 'head_model=FCN',
                                'x': 0.74,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {},
                                'showarrow': False,
                                'text': 'calibration=finetuning',
                                'textangle': 90,
                                'x': 0.98,
                                'xanchor': 'left',
                                'xref': 'paper',
                               

### IEEEPPG Results

In [ ]:
ieeeppg_result = pd.read_csv('./Pos_training/ieeeppg_results.csv')
ieeeppg_result.head()

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,mse,mse_std,r2_score,r2_score_std,rmse,rmse_std,mae,mae_std
0,TSTCC,MLP,linearprobing,one_domain,IEEEPPG,517.3157,0.0101,0.0043,22.7445,0.0498,18.3980,0.1768,529.4080,1.4312
1,TSTCC,MLP,linearprobing,four_domains,IEEEPPG,518.3404,0.0082,0.0030,22.7671,0.0346,18.4338,0.1247,529.0043,1.1918
2,TSTCC,MLP,finetuning,one_domain,IEEEPPG,535.3663,-0.0244,0.0264,23.1364,0.2993,19.1758,0.5177,543.2826,11.4943
3,TSTCC,MLP,finetuning,four_domains,IEEEPPG,604.8880,-0.1574,0.0264,24.5932,0.2804,21.0037,0.3759,591.9205,15.9592
4,TSTCC,FCN,linearprobing,one_domain,IEEEPPG,1190.1912,-1.2774,0.2310,34.4638,1.7447,30.7696,1.6269,1177.3956,119.0651


In [ ]:
fig = px.bar(ieeeppg_result, x='ssl_model', y='rmse', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group')

fig.update_layout(height=640, width=1000, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

In [ ]:
ieeeppg_result = pd.read_csv('./Pos_training/ieeeppg_results.csv')
ieeeppg_result.head()

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,mse,mse_std,r2_score,r2_score_std,rmse,rmse_std,mae,mae_std
0,TSTCC,MLP,linearprobing,one_domain,IEEEPPG,517.3157,0.0101,0.0043,22.7445,0.0498,18.3980,0.1768,529.4080,1.4312
1,TSTCC,MLP,linearprobing,four_domains,IEEEPPG,518.3404,0.0082,0.0030,22.7671,0.0346,18.4338,0.1247,529.0043,1.1918
2,TSTCC,MLP,finetuning,one_domain,IEEEPPG,535.3663,-0.0244,0.0264,23.1364,0.2993,19.1758,0.5177,543.2826,11.4943
3,TSTCC,MLP,finetuning,four_domains,IEEEPPG,604.8880,-0.1574,0.0264,24.5932,0.2804,21.0037,0.3759,591.9205,15.9592
4,TSTCC,FCN,linearprobing,one_domain,IEEEPPG,1190.1912,-1.2774,0.2310,34.4638,1.7447,30.7696,1.6269,1177.3956,119.0651


In [ ]:
fig = px.bar(ieeeppg_result, x='ssl_model', y='rmse', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group')

fig.update_layout(height=640, width=1000, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)